# Radioactive Decay Energy

Within the ejecta of a supernova, the $\gamma$-rays largely come from the decay of $^{56}Ni$ into $^{56}Co$, which releases a significant amount of energy. 

When $^{56}Ni$ decays into $^{56}Co$ it can release a $\gamma$-ray at several different transition levels. Each transition level has an energy and an associated probability out of 100 decays. For example, the transition from Energy level 9 to Energy level 7 has an energy of 0.270 Mev and a probability of 36.5 out of 100 decays. To find the total energy per decay you multipliy each energy with its associated probability and add them all up.

In [2]:
import numpy as np
import pandas as pd
from astropy import units as u
from astropy import constants as const

from tardis.energy_input.gamma_ray_channel import create_isotope_dicts
from tardis.energy_input.gamma_ray_channel import create_inventories_dict
from tardis.energy_input.gamma_ray_channel import calculate_total_decays
from tardis.energy_input.gamma_ray_channel import create_isotope_decay_df
from tardis.energy_input.energy_source import get_nuclear_lines_database
from tardis.io.atom_data import download_atom_data
from tardis.io.atom_data import AtomData

/home/ceceliapowers/software/tardis/tardis/__init__.py:23: UserWarning: Astropy is already imported externally. Astropy should be imported after TARDIS.
  warnings.warn(


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [3]:
# energies of each transition
t_energies = np.array([0.270, 0.750, 0.480, 1.56, 0.812, 0.158]) * u.MeV
# probabilities of each transition
t_prob = np.array([.365, .495, .366, .140, .860, 1.00])

energy_per_decay = sum(t_energies * t_prob)
energy_per_decay

<Quantity 1.7202 MeV>

From the above cell, we get the energy per transition of 1.72 MeV. Note that this comes from a simplified scheme of energies and the real total energy per $^{56}Ni$ decay we use is 1.75 MeV. <strong data-cite="1994Nadyozhin">[]</strong> 

The $^{56}Co$ produced from the decay of $^{56}Ni$ is also radioactive and will decay into $^{56}Fe$ and release more $\gamma$-rays, however this decay is more complicated than the decay of $^{56}Ni$. Whereas $^{56}Ni$ only decays through electron capture, $^{56}Co$ can decay either by electron capture, which occurs for 81 out of 100 cases, or through positron decay, which occurs for 19 out of 100 cases.

Positron decay produces positrons with a given kinetic energy, that will eventually annihilate with electrons to produce two 0.511 MeV $\gamma$-rays. The scheme of decays for $^{56}Co$ is slightly more complicated than the $^{56}Ni$ scheme, but to find the total energy per decay, you follow the same process. The total energy per decay from $\gamma$-rays is 3.61 MeV and the total kinetic energy of positrons is 0.12 MeV

 The total rate of energy production for a mass of $^{56}Ni$ at a given time is given by the following equation:

$$\epsilon = \frac{M_\odot}{56m_{u}}\frac{1}{\tau_{\text{Co}}-\tau_{\text{Ni}}}[[Q_{\text{Ni}}(\frac{\tau_{\text{Co}}}{\tau_{\text{Ni}}}-1)-Q_{\text{Co}}]\exp(-t/\tau_{\text{Ni}})+Q_{\text{Co}}\exp(-t/\tau_{\text{Co}})]\frac{M_{Ni0}}{M_\odot}$$

$M_\odot$ is a solar mass. $56_{u}$ is 56 atomic mass units. 

$\tau_{Ni}$ is the lifetime of $^{56}Ni$ which is 8.80 days and $\tau_{Co}$ is the lifetime of $^{56}Co$ which is 111.3 days. 

$Q_{\text{Ni}}$ is the energy per decay of $^{56}Ni$ which is 1.75 MeV and $Q_{\text{Co}}$ is the sum of the energy per decay pf $^{56}Co$ from $\gamma$-rays and the kinetic energy from positrons which is 3.73 MeV

If we plug these values into the equation we get the equation:

$$\epsilon = (6.45e43\exp(-t/8.8)+1.45e43\exp(-t/111.3))\frac{M_{Ni0}}{M_\odot} erg/s$$

In [4]:
#time in days
time = 10 * u.day
#mass of Ni56 in solar masses
m_Ni56 = 1 * const.M_sun

energy_production_rate = (6.45e43 * np.exp(-time/(8.8*u.day)) + 1.45e43 * np.exp(-time/(111.3*u.day))) * (m_Ni56/ const.M_sun) * u.erg /u.s

print(f"The total energy production rate for 1 solar mass of 56Ni after 10 days is: {energy_production_rate:.2e}")

The total energy production rate for 1 solar mass of 56Ni after 10 days is: 3.40e+43 erg / s


In [5]:
total_energy_production = 1.885e50 * (m_Ni56/const.M_sun) * u.erg
print(f"The total energy production for 1 solar mass of 56Ni is: {total_energy_production}")

The total energy production for 1 solar mass of 56Ni is: 1.885e+50 erg


Next we show how we can find the same result from functions in TARDIS-HE

First we create a DataFrame with isotope abundances. This DataFrame typically comes from a model, but for our purposes we create an ejecta with 5 shells, made entirely of $^{56}Ni$.

In [6]:
mass_fractions = {0:1,
                  1:1,
                  2:1,
                  3:1,
                  4:1}

raw_isotope_abundance = pd.DataFrame(mass_fractions, index=pd.MultiIndex.from_tuples([(28, 56)], names=["atomic_number", "mass_number"]))
raw_isotope_abundance

,,0,1,2,3,4
atomic_number,mass_number,,,,,
28,56,1,1,1,1,1


Then we create an array of shell masses. We want to find the total energy production of 1 solar mass of $^{56}Ni$, so we give each of the five shells a mass of $\frac{1}{5}M_\odot$

In [7]:
shell_masses = np.ones(5)*0.2*const.M_sun.to(u.g)
shell_masses

<Quantity [3.97681974e+32, 3.97681974e+32, 3.97681974e+32, 3.97681974e+32,
           3.97681974e+32] g>

Now we can use TARDIS-HE functions to get a DataFrame with the total number of decays and decay energy.

In [8]:
isotope_dict = create_isotope_dicts(raw_isotope_abundance, shell_masses)
isotope_dict

{0: {'Ni56': 3.976819741396102e+32},
 1: {'Ni56': 3.976819741396102e+32},
 2: {'Ni56': 3.976819741396102e+32},
 3: {'Ni56': 3.976819741396102e+32},
 4: {'Ni56': 3.976819741396102e+32}}

In [9]:
inventories = create_inventories_dict(isotope_dict)
inventories

{0: Inventory activities (Bq): {'Ni-56': 5.653446204017355e+48}, decay dataset: icrp107_ame2020_nubase2020,
 1: Inventory activities (Bq): {'Ni-56': 5.653446204017355e+48}, decay dataset: icrp107_ame2020_nubase2020,
 2: Inventory activities (Bq): {'Ni-56': 5.653446204017355e+48}, decay dataset: icrp107_ame2020_nubase2020,
 3: Inventory activities (Bq): {'Ni-56': 5.653446204017355e+48}, decay dataset: icrp107_ame2020_nubase2020,
 4: Inventory activities (Bq): {'Ni-56': 5.653446204017355e+48}, decay dataset: icrp107_ame2020_nubase2020}

In [10]:
cumulative_decay_df = calculate_total_decays(inventories, 500*u.day)
print(cumulative_decay_df)

                      number_of_decays
shell_number isotope                  
0            Ni56         4.281026e+54
             Co56         4.228761e+54
1            Ni56         4.281026e+54
             Co56         4.228761e+54
2            Ni56         4.281026e+54
             Co56         4.228761e+54
3            Ni56         4.281026e+54
             Co56         4.228761e+54
4            Ni56         4.281026e+54
             Co56         4.228761e+54


In [11]:
download_atom_data('kurucz_cd23_chianti_H_He')
atom_data_file = 'kurucz_cd23_chianti_H_He.h5'
atom_data = AtomData.from_hdf(atom_data_file)
gamma_ray_lines = atom_data.decay_radiation_data

INFO:tardis.io.atom_data.util:
	Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path.
	Exists in TARDIS Data repo /home/ceceliapowers/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5
INFO:tardis.io.atom_data.base:Reading Atom Data with: UUID = 6f7b09e887a311e7a06b246e96350010 MD5  = 864f1753714343c41f99cb065710cace 
INFO:tardis.io.atom_data.base:Non provided Atomic Data: synpp_refs, photoionization_data, yg_data, two_photon_data, linelist_atoms, linelist_molecules


In [12]:
isotope_decay_df = create_isotope_decay_df(cumulative_decay_df, gamma_ray_lines)
isotope_decay_df

number_of_decays decay_mode radiation  \
shell_number isotope                                          
0            Ni56         4.281026e+54         EC        bp   
             Ni56         4.281026e+54         EC        bp   
             Ni56         4.281026e+54         EC        bp   
             Ni56         4.281026e+54         EC     bp av   
             Ni56         4.281026e+54         EC         e   
...                                ...        ...       ...   
4            Co56         4.228761e+54         EC         g   
             Co56         4.228761e+54         EC         g   
             Co56         4.228761e+54         EC         g   
             Co56         4.228761e+54         EC         g   
             Co56         4.228761e+54         EC         g   

                      radiation_energy_keV  radiation_intensity  \
shell_number isotope                                              
0            Ni56                   65.000              0.00060   
             Ni56                  408.000              0.00003   
             Ni56                  478.000              0.00003   
             Ni56                  100.000              0.00070   
             Ni56                    6.070             54.50000   
...                                    ...                  ...   
4            Co56                 3369.860              0.01010   
             Co56                 3451.232              0.94900   
             Co56                 3548.050              0.19550   
             Co56                 3600.800              0.01670   
             Co56                 3611.530              0.00860   

                      energy_per_channel_keV  decay_energy_keV  \
shell_number isotope                                             
0            Ni56                   0.000390      1.669600e+51   
             Ni56                   0.000122      5.239975e+50   
             Ni56                   0.000143      6.138991e+50   
             Ni56                   0.000700      2.996718e+51   
             Ni56                   3.308150      1.416227e+55   
...                                      ...               ...   
4            Co56                   0.340356      1.439284e+54   
             Co56                  32.752192      1.385012e+56   
             Co56                   6.936438      2.933254e+55   
             Co56                   0.601334      2.542896e+54   
             Co56                   0.310592      1.313417e+54   

                      decay_energy_erg  
shell_number isotope                    
0            Ni56         2.674994e+42  
             Ni56         8.395366e+41  
             Ni56         9.835748e+41  
             Ni56         4.801271e+42  
             Ni56         2.269047e+46  
...                                ...  
4            Co56         2.305986e+45  
             Co56         2.219034e+47  
             Co56         4.699590e+46  
             Co56         4.074168e+45  
             Co56         2.104327e+45  

[485 rows x 8 columns]

Finally we can sum up the decay energy column and find that it gives a very similar result to the analytical solution above.

In [13]:
sum(isotope_decay_df["decay_energy_erg"])

1.8965311701805456e+50